## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
from os.path import expanduser

___
### Add to sys.path

Within the barchart_api module, references like:

```from barchartapi import barchart_api```

will not work

In [2]:
WORKSPACE_DIR = '../..' 
PROJECT_DIR = f'{WORKSPACE_DIR}/barchartapi' 
WORKING_DIR = f'{PROJECT_DIR}/barchartapi' 
if WORKING_DIR not in sys.path:
    sys.path.append(WORKING_DIR)
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)
if WORKSPACE_DIR not in sys.path:
    sys.path.append(WORKSPACE_DIR)
import barchart_api as bcapi

___
### Create paths to where you will store the history data

In [3]:
# DROPBOX_MARKET_DATA_DIR = expanduser('~/Dropbox/market_data')
# STOCKS_DIR = f'{DROPBOX_MARKET_DATA_DIR}/stocks'
# !ls {STOCKS_DIR}
STOCKS_DIR = f'./temp_folder/stocks'
try:
    os.makedirs(STOCKS_DIR)
except:
    pass

___
### Create an instance of BcHist
1. must set bar_type to minutes, hour or daily
2. must set interval
 * for minutes, set interval to 1, 5, 15, 30 or 60
 * for other types, see https://www.barchart.com/ondemand/api/getHistory

In [4]:
# set this to 'free' or 'paid'
endpoint = 'free' # free or paid

# set the bar_type and the interval
bar_type='minutes' # minutes, daily, monthly
interval=30 # 1,5,15,30,60

# create an instance 
api_key = open(f'./temp_folder/{endpoint}_api_key.txt','r').read()
endpoint_type=f'{endpoint}_url'
bch = bcapi.BcHist(api_key, bar_type=bar_type, interval=interval,endpoint_type = endpoint_type)


Barchart OnDemand Client: https://marketdata.websol.barchart.com/


___
### Main Loop to fetch history data from Barchart

#### Inputs
On each call to BcHist.get_history, provide:
 * a beginning yyyymmdd integer
 * an ending yyyymmdd integer
 * a short name like:
  * a stock symbol like: SPY,USO,IBM, etc
  * a futures symbol like: CLJ18, GCG19, ESH17, etc

#### Outputs
The get_history method will return a tuple, where:
 * tup[0] provides a status

* tup[1] provides a pandas DataFrame of data, or None

In [5]:
#  set a date range
beg_yyyymmdd = 20181201
end_yyyymmdd = 20190109

# set up stocks to get
#   below are some example lists of stocks
spydr_short_names = ['XLE','XLU','XLK','XLB','XLP','XLY','XLI','XLC','XLV','XLF']
commodity_etf_short_names = ['USO','UNG','DBC','DBA','GLD','USCI']
currency_etf_short_names = ['FXY','FXE','FXB','FXF','FXC','FXA']
my_portfolio_short_names = ['XLE','SPY','USO','GLD','XLU']

# set short_names to the desired list to get
short_names = my_portfolio_short_names

for short_name in short_names:
    sn_path = f'{STOCKS_DIR}/{short_name}.csv'
    if os.path.isfile(sn_path):
        print(f'BYPASSING: {short_name}')
        continue
    print(f'get_history: {short_name} BEGIN {datetime.datetime.now()}')
    tup = bch.get_history(short_name, beg_yyyymmdd, end_yyyymmdd)
    print(f'get_history: {short_name} WRITING DATA {datetime.datetime.now()}')
    tup[1].to_csv(sn_path,index=False)
    print(f'get_history: {short_name} END {datetime.datetime.now()}')
    

BYPASSING: XLE
BYPASSING: SPY
BYPASSING: USO
BYPASSING: GLD
BYPASSING: XLU


___
### Do something interesting with the returned data

In [6]:
df_portfolio = None
for short_name in short_names:
    sn_path = f'{STOCKS_DIR}/{short_name}.csv'
    df_temp = pd.read_csv(sn_path)
    df_temp = df_temp[['timestamp','close']]
    df_temp = df_temp.rename(columns={'close':f'{short_name}'})
    if df_portfolio is None:
        df_portfolio = df_temp.copy()
    else:
        df_portfolio = df_portfolio.merge(df_temp,how='inner',on='timestamp')
df_corr = df_portfolio.corr()
df_corr

,XLE,SPY,USO,GLD,XLU
XLE,1.000000,0.976647,0.960263,-0.562698,0.768358
SPY,0.976647,1.000000,0.920393,-0.616033,0.818876
USO,0.960263,0.920393,1.000000,-0.643154,0.794815
GLD,-0.562698,-0.616033,-0.643154,1.000000,-0.894643
XLU,0.768358,0.818876,0.794815,-0.894643,1.000000


In [7]:
fn = f'{STOCKS_DIR}/{"_".join(df_corr.columns.values)}.csv'
df_corr.to_csv(fn,index=False)

In [8]:
df_portfolio.head()

,timestamp,XLE,SPY,USO,GLD,XLU
0,2018-12-10T09:30:00-05:00,62.56477,260.1508,10.89,117.88,55.56803
1,2018-12-10T10:00:00-05:00,62.53505,260.8269,10.97,117.86,55.70189
2,2018-12-10T10:30:00-05:00,61.80191,258.9801,10.95,117.74,55.33005
3,2018-12-10T11:00:00-05:00,61.37589,258.1525,10.91,117.89,55.29039
4,2018-12-10T11:30:00-05:00,61.49478,258.7490,10.93,117.83,55.44904


In [9]:
cols = list(set(list(df_portfolio.columns.values))-set(['timestamp']))
bars_per_day = 8*2
perc_of_day = 1/bars_per_day
perc_of_year = perc_of_day/256
df_portfolio[cols].pct_change().iloc[1:].std()/perc_of_year**.5

GLD    0.122897
XLE    0.368310
SPY    0.304023
USO    0.521594
XLU    0.239980
dtype: float64

## END